# Imports

In [ ]:
import os, sys
sys.path.append(os.path.abspath('../../two_dim_majoranas/'))

import hpc05
import holoviews as hv

import numpy as np
import matplotlib.pyplot as plt
import kwant
import scipy.constants
import cmath
import functools as ft
import adaptive
adaptive.notebook_extension()

import sns_system, spectrum, topology, supercurrent, formulas

In [ ]:
def abs_min_log_loss(xs, xy_scale, data):
    from adaptive.learner.learner1D import default_loss
    data = {k: np.log(np.abs(v).min()) for k, v in data.items()}
    return default_loss(xs, xy_scale, data)

coloring =  {'middle_interior' : 'grey',
             'middle_barrier' : 'black',
             'bottom_superconductor' : 'gold',
             'top_superconductor' : 'gold',
             'top_cut' : 'red',
             'bottom_cut' : 'blue'}

In [ ]:
constants = dict(
    m_eff=0.02 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

# Bandstructure normal system

In [ ]:
syst_pars = dict(
    L_m = 200, L_x = 10, L_sc_up = 200, L_sc_down = 200,
    z_x = 800, z_y = 0, a   = 10,
    parallel_curve = False,
    sawtooth=True,
    transverse_soi = True,
    mu_from_bottom_of_spin_orbit_bands = True,
    k_x_in_sc = True, wraparound = True,
    current = False, ns_junction=False)

params_raw= dict(g_factor_middle = 26,
                 g_factor_left = 0,
                 g_factor_right = 0,
                 mu = 10,
                 alpha_middle = 20,
                 alpha_left = 0,
                 alpha_right = 0,
                 Delta_left = 2,
                 Delta_right = 2,
                 B = 1,
                 phase = np.pi,
                 T = 0.0,
                 V = 0)

params = dict(**constants,
              **params_raw)


syst, site_colors, _ = sns_system.make_system(**syst_pars)

try:
    for k,v in site_colors.items():
        if site_colors.get(k) is not None:
            site_colors[k] = coloring[v]
except:
    pass
        
kwant.plot(syst,site_color=sns_system.site_color_function(site_colors, syst), dpi=100);

In [ ]:
def bands_wrap(k):
    ham = syst.hamiltonian_submatrix(sparse=True, params=dict(**params, k_x=k))
    e, ev = spectrum.sparse_diag(ham, 100, 0)
    return np.sort(e)
    
learner = adaptive.Learner1D(bands_wrap, [0, np.pi], abs_min_log_loss)


In [ ]:
runner = adaptive.Runner(learner, ntasks=48)
runner.live_info()

In [ ]:
kf = formulas.fermi_wavenumber(params['mu'], 0.02)*syst_pars['a']
emax = np.max(learner.data.values())
learner.plot().redim(x='k', y='E')*hv.Path(([kf, kf],[-emax,emax]))

# Why does minimum of energy occur around $k_x = k_F$

To keep the kinetic term in the hamiltonian as low as possible, one generally should minimize the magnitude of $\vec{k}$. 
The lowest possible accesible mode is: $\rvert\vec{k}\rvert = k_F$. Of course this leaves us to choose  the angle of $\vec{k}$. Which angle has the lowest energy?

# Snake like system

In [ ]:
sp_flat = syst_pars.copy()
sp_flat['z_y'] = 120
sp_flat['L_x'] = 800
sp_flat['sawtooth'] = False
sp_flat['parallel_curve'] = True

params = dict(**constants,
              **params_raw)

syst, site_colors, _ = sns_system.make_system(**sp_flat)

try:
    for k,v in site_colors.items():
        if site_colors.get(k) is not None:
            site_colors[k] = coloring[v]
except:
    pass
        
kwant.plot(syst,site_color=sns_system.site_color_function(site_colors, syst), dpi=100);

In [ ]:
def bands_wrap(k):
    ham = syst.hamiltonian_submatrix(sparse=True, params=dict(**params, k_x=k))
    e, ev = spectrum.sparse_diag(ham, 100, 0)
    return np.sort(e)
    
learner = adaptive.Learner1D(bands_wrap, [0, np.pi], abs_min_log_loss)


In [ ]:
runner = adaptive.Runner(learner, ntasks=48)
runner.live_info()

In [ ]:
kf = formulas.fermi_wavenumber(params['mu'], 0.02)*syst_pars['a']
emax = np.max(learner.data.values())
learner.plot().redim(x='k', y='E')*hv.Path(([kf, kf],[-emax,emax]))

# Wavefunction

In [ ]:
sp_flat = syst_pars.copy()
sp_flat['z_y'] = 0*150
sp_flat['L_x'] = 2400
sp_flat['wraparound'] = False
sp_flat['parallel_curve'] = False
sp_flat['sawtooth'] = True

params = dict(**constants,
              **params_raw)

syst, site_colors, _ = sns_system.make_system(**sp_flat)

try:
    for k,v in site_colors.items():
        if site_colors.get(k) is not None:
            site_colors[k] = coloring[v]
except:
    pass

In [ ]:
ham = syst.hamiltonian_submatrix(params=params, sparse=True)
d = kwant.operator.Density(syst)
e, ev = spectrum.sparse_diag(ham,4,0)

In [ ]:
e

In [ ]:
ax=plt.axes();
kwant.plot(syst,site_color=sns_system.site_color_function(site_colors, syst), dpi=100, ax=ax);
kwant.plotter.map(syst, d(ev[:,0]), ax=ax)
# plt.savefig('no_ohase_difference', dpi=1000)

In [ ]:
ax=plt.axes();
kwant.plot(syst,site_color=sns_system.site_color_function(site_colors, syst), dpi=100, ax=ax);
kwant.plotter.map(syst, d(ev[:,0]), ax=ax)
# plt.savefig('no_ohase_difference', dpi=1000)